# Vessel Route Optimization

In [2]:
import numpy as np
from numpy import random

## Semi-random vectors generation

In [3]:
wind_degrees = []
days = 10
max_change = 0.3
#random.seed(1234)

for day in range(days):
    if day == 0:
        wind_degrees.append(random.randint(0,360))
    else:
        last_day = wind_degrees[day-1]
        wind_degrees.append(random.randint(last_day*(1-max_change),last_day*(1+max_change)))
        
wind_speed = list(random.randint(0,28,(1,days)).flatten())

print("Wind direction (degrees):", wind_degrees)
print("Wind speed (m/s):", wind_speed)

Wind direction (degrees): [316, 367, 339, 240, 294, 221, 171, 124, 142, 108]
Wind speed (m/s): [14, 18, 15, 22, 20, 27, 23, 8, 23, 6]


## Multiple array problem

In [3]:
def random_space(max_value, days=5, positions=1, day_max_change=0.3, position_max_change=0.2, seed=None):    
    random.seed(seed)
    degrees_matrix = []
    wind_degrees = []
    
    for day in range(days):
        wind_degrees.append(random.randint(1,max_value))
    degrees_matrix.append(wind_degrees) 

    for position in range(0,positions-1):
        position_degree = []
        for day in range(days):
            same_day_other_position = abs(degrees_matrix[position-1][day])  
            if same_day_other_position != 0:
                position_degree.append(random.randint(same_day_other_position*(1-position_max_change),same_day_other_position*(1+position_max_change)))
            else: 
                position_degree.append(random.randint(1,max_value))
        degrees_matrix.append(position_degree)
    
    return degrees_matrix

days = 5
positions = 3
day_max_change = 0.3
position_max_change = 0.3
seed = 126

wind_direction = random_space(360, days, positions, day_max_change, position_max_change)
wind_speed = random_space(27, days, positions, day_max_change, position_max_change)

print("Wind direction (degrees):", wind_direction)
print("Wind speed (m/s):", wind_speed)

Wind direction (degrees): [[144, 24, 76, 180, 336], [118, 29, 63, 217, 433], [117, 30, 70, 146, 410]]
Wind speed (m/s): [[10, 4, 19, 21, 1], [7, 4, 17, 22, 0], [7, 2, 19, 19, 0]]


In [4]:
#CONSTANTS
min_wind_angle = 20 
max_wind_angle = 160 
min_wind_strength = 20

def open_sail(wind_angle, wind_strength):
    open_sail_bool = False 
    
    if wind_angle > min_wind_angle and wind_angle < max_wind_angle and wind_strength > min_wind_strength :
        open_sail_bool = True


    return open_sail_bool


print("We open the sail : ", open_sail(80,40))

We open the sail :  True


In [42]:
f(-123.23182855)

9.391838418082443e-10

In [43]:
f(360-123.23182855)

9.391634137045912e-10